In [1]:
# generate user-item matrix
import pandas

user_loc_data = pandas.read_csv("generated_data/cleaned_checkins.csv")
print(user_loc_data.head())

   userid  placeid
0    1338   482954
1    1338   365256
2    1338    89504
3    1338   365256
4    1338    39597


In [2]:
from scipy import sparse

user_list = list(user_loc_data['userid'])
locations_list = list(user_loc_data['placeid'])
user_locations = list(zip(user_list,locations_list))
user_item_dict ={}

# create user-index mapping
user_idx = {}
user_set = list(set(user_list))
user_set.sort()
idx = 0
for u in user_set:
    user_idx[u] = idx
    idx += 1
    
# create location-index mapping
loc_idx = {}
loc_set = list(set(locations_list))
loc_set.sort()
idx = 0
for l in loc_set:
    loc_idx[l] = idx
    idx += 1
    
# compute the user/item counts
for user_loc in user_locations:
    if user_loc in user_item_dict:
        user_item_dict[user_loc] += 1
    else:
        user_item_dict[user_loc] = 1
        
# create data,i,j for sparse matrix
data = []
i = []
j = []
for user_loc in user_item_dict:
    userid = user_loc[0]
    locid = user_loc[1]
    count = user_item_dict[user_loc]
    i.append(user_idx[userid])
    j.append(loc_idx[locid])
    data.append(count)
    
# create user-item sparse matrix
user_item_sparse = sparse.coo_matrix((data,(i,j)))

In [ ]:
user_item_df = pandas.DataFrame(user_item_sparse.toarray(),index=user_set,columns=loc_set)
user_item_df.head()

In [4]:
import numpy

sparse.save_npz('generated_data/user_item_matrix.npz', user_item_sparse)
numpy.savez('generated_data/user_item_matrix_columns.npz',user_set,loc_set)

In [5]:
#uis = sparse.load_npz('generated_data/user_item_matrix.npz')
#user_loc = numpy.load('generated_data/user_item_matrix_columns.npz')
#u = list(user_loc['arr_0'])
#l = list(user_loc['arr_1'])